In [2]:
import numpy as np
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import pandas as pd

In [3]:
var_names = ["demand", "trans", "storage", "inertia", "solar"]
regression_out = pd.DataFrame(index=pd.RangeIndex(stop=((len(var_names)**2-len(var_names)))//2), columns=["x", "y", "bx", "by", "bxy", "a", "R^2"])

In [3]:
# Create model 
model = Pipeline([("poly", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
                  ("regression", LinearRegression())])

In [4]:
row = 0

for i in range(len(var_names)):
    for j in range(i+1, len(var_names)):
        # Get data from Normalized Data folder
        data = np.loadtxt("../Data/Normalized Data/" + var_names[i] + "-" + var_names[j] + ".csv",delimiter=",",skiprows=1)
        regression_out.loc[row,["x","y"]] = [var_names[i], var_names[j]] # Set first two columns of DataFrame to names of variables

        # Process and perform regression on data
        x_data = data[:,:-1]
        y_data = data[:,-1].T
        model = model.fit(x_data, y_data)

        # Store regression output in regression_out DataFrame
        regression_out.loc[row, ["bx", "by", "bxy"]] = model["regression"].coef_
        regression_out.loc[row, "a"] = model["regression"].intercept_
        regression_out.loc[row, "R^2"] = model.score(x_data, y_data)

        row += 1

regression_out

,x,y,bx,by,bxy,a,R^2
0,demand,trans,-1.957281,0.679595,-9.340901,0.210968,0.946422
1,demand,storage,-2.166828,-1.34159,50.521534,0.208434,0.799404
2,demand,inertia,-1.908856,-0.052164,1.651915,0.203879,0.738288
3,demand,solar,-2.426346,-1.778063,65.564525,0.213773,0.880668
4,trans,storage,0.775969,-1.326959,-14.561935,0.202406,0.972201
5,trans,inertia,0.828415,-0.049131,-0.747713,0.193451,0.930992
6,trans,solar,0.679419,-1.797345,-5.842316,0.2084,0.961667
7,storage,inertia,-1.011596,-0.045119,1.045347,0.195133,0.704626
8,storage,solar,-1.222305,-1.477091,35.159862,0.202681,0.805526
9,inertia,solar,-0.048894,-1.354172,1.398022,0.198746,0.744912


In [5]:
regression_out.to_csv("regression-output.csv", index=False)

In [17]:
# Redefine regression_out to store regression coefficients for parameters when varied independently
regression_out = pd.DataFrame(index=pd.RangeIndex(stop=len(var_names)), columns=["x", "bx", "a", "R^2"])
regression_out["x"] = var_names
regression_out

,x,bx,a,R^2
0,demand,NaN,NaN,NaN
1,trans,NaN,NaN,NaN
2,storage,NaN,NaN,NaN
3,inertia,NaN,NaN,NaN
4,solar,NaN,NaN,NaN


In [18]:
model = LinearRegression()

In [21]:
row = 0

# Perform univariate regression on demand response
i = 0
j = 1
# Get data from Normalized Data folder
data = np.loadtxt("../Data/Normalized Data/" + var_names[i] + "-" + var_names[j] + ".csv",delimiter=",",skiprows=1)

# Process and perform regression on data
x_data = data[::5,0].reshape(-1,1)
y_data = data[::5,-1].T
model = model.fit(x_data, y_data)

# Store regression output in regression_out DataFrame
regression_out.loc[row, "bx"] = model.coef_[0]
regression_out.loc[row, "a"] = model.intercept_
regression_out.loc[row, "R^2"] = model.score(x_data, y_data)

row += 1


# Perform univariate regression on the other 4 parameters
for j in range(1, len(var_names)):
    # Get data from Normalized Data folder
    data = np.loadtxt("../Data/Normalized Data/" + var_names[i] + "-" + var_names[j] + ".csv",delimiter=",",skiprows=1)

    # Process and perform regression on data
    x_data = data[-5:,1].reshape(-1,1)
    y_data = data[-5:,-1].T
    model = model.fit(x_data, y_data)

    # Store regression output in regression_out DataFrame
    regression_out.loc[row, "bx"] = model.coef_[0]
    regression_out.loc[row, "a"] = model.intercept_
    regression_out.loc[row, "R^2"] = model.score(x_data, y_data)

    row += 1

regression_out


,x,bx,a,R^2
0,demand,-2.176816,0.218907,0.999914
1,trans,0.744551,0.209213,0.93021
2,storage,-1.348929,0.209596,0.873116
3,inertia,-0.051099,0.204301,0.7394
4,solar,-1.773236,0.214461,0.944124


In [20]:
regression_out.to_csv("individual-regression-output.csv", index=False)